# 01 - Download and Organize Data

This notebook collates the RNA-seq gene count tables previously downloaded from the GDC portal, cleans a combined expression matrix, and saves the organized outputs under `data/processed/` for downstream analysis.


**Workflow overview**

1. Discover every `.tsv` expression file that lives inside the raw GDC download bundles (ignoring manifest `.txt` files).
2. Build a tidy manifest so we can trace each file back to its case identifier.
3. Load the TPM counts, align the genes across all samples, and write a single expression matrix that other notebooks can use.


In [2]:
from __future__ import annotations

import sys
from pathlib import Path

import pandas as pd


def find_project_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / 'README.md').exists():
            return candidate
    raise FileNotFoundError('Unable to locate repository root (README.md not found)')


PROJECT_ROOT = find_project_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import ProjectConfig

config = ProjectConfig()
RAW_DATA_DIR = config.raw_data_dir / 'star gene counts'
PROCESSED_DATA_DIR = config.processed_data_dir
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

RAW_DATA_DIR, PROCESSED_DATA_DIR


(PosixPath('/Users/lennonmccartney/Desktop/tcga-brca-multiomics-subtyping/data/raw/star gene counts'),
 PosixPath('/Users/lennonmccartney/Desktop/tcga-brca-multiomics-subtyping/data/processed'))

## Discover downloaded expression bundles

Each download bundle from GDC sits in its own UUID-named directory. We walk the tree, capture every `.tsv` file (ignoring plain `.txt` documents such as manifests), and track minimal metadata for reproducibility.


In [3]:
import pandas as pd


def collect_expression_files(root: Path) -> pd.DataFrame:
    '''Return metadata for every RNA-seq expression TSV found under ``root``.'''

    records: list[dict[str, object]] = []
    bundle_dirs = sorted(p for p in root.iterdir() if p.is_dir())
    for bundle in bundle_dirs:
        for path in sorted(bundle.glob('*.tsv')):
            if path.name.lower().endswith('.txt'):
                continue
            sample_id = path.name.split('.')[0]
            relative = path.relative_to(root)
            case_id = bundle.name
            records.append(
                {
                    'sample_id': sample_id,
                    'case_id': case_id,
                    'file_name': path.name,
                    'path': path,
                    'relative_path': relative.as_posix(),
                }
            )
    if not records:
        raise FileNotFoundError(f'No TSV files discovered under {root}')
    return pd.DataFrame.from_records(records)


expression_index = collect_expression_files(RAW_DATA_DIR)
print(f'Discovered {len(expression_index)} expression TSV files.')
expression_index.head()


Discovered 1197 expression TSV files.


,sample_id,case_id,file_name,path,relative_path
0,ba295155-272e-43eb-9d6a-e4c9c392e68b,0019c951-16c5-48d0-85c8-58d96b12d330,ba295155-272e-43eb-9d6a-e4c9c392e68b.rna_seq.a...,/Users/lennonmccartney/Desktop/tcga-brca-multi...,0019c951-16c5-48d0-85c8-58d96b12d330/ba295155-...
1,8d1641ea-7552-4d23-9298-094e0056386a,0022cd20-f64f-4773-b9ff-a3de0b71b259,8d1641ea-7552-4d23-9298-094e0056386a.rna_seq.a...,/Users/lennonmccartney/Desktop/tcga-brca-multi...,0022cd20-f64f-4773-b9ff-a3de0b71b259/8d1641ea-...
2,2f51534b-248b-4999-bc3f-e42a2e98332e,00469928-b243-4cae-acd7-134508e99ceb,2f51534b-248b-4999-bc3f-e42a2e98332e.rna_seq.a...,/Users/lennonmccartney/Desktop/tcga-brca-multi...,00469928-b243-4cae-acd7-134508e99ceb/2f51534b-...
3,b321a3f5-043d-42c6-8c9d-5784d45cb85c,0081f507-b104-4214-9ea1-31dd69130991,b321a3f5-043d-42c6-8c9d-5784d45cb85c.rna_seq.a...,/Users/lennonmccartney/Desktop/tcga-brca-multi...,0081f507-b104-4214-9ea1-31dd69130991/b321a3f5-...
4,cafc9e36-c5f0-45df-ad03-16210ff0d870,0094f9d0-45ec-4aad-bca0-71c60bdd7113,cafc9e36-c5f0-45df-ad03-16210ff0d870.rna_seq.a...,/Users/lennonmccartney/Desktop/tcga-brca-multi...,0094f9d0-45ec-4aad-bca0-71c60bdd7113/cafc9e36-...


## Helper to read a single expression table

We only need the `gene_id` column plus a quantitative abundance metric (`tpm_unstranded`). The helper below returns one `pd.Series` per sample and removes the technical summary rows (the `N_*` counters at the top of each file).


In [4]:
from pathlib import Path


def read_expression_table(path: Path, value_column: str = "unstranded") -> pd.Series:
    '''Load one RNA-seq table and return the chosen value column indexed by gene ID.'''

    usecols = ["gene_id", value_column]
    df = pd.read_csv(
        path,
        sep="\t",
        comment="#",
        usecols=usecols,
        dtype={value_column: "float32"},
    ).dropna(subset=["gene_id"])
    series = df.set_index("gene_id")[value_column]
    series = series[~series.index.str.startswith("N_")]
    series.name = path.name.split(".")[0]
    return series


# Quick sanity check on the first file
first_sample = read_expression_table(Path(expression_index.loc[0, "path"]))
first_sample.head()


gene_id
ENSG00000000003.15    4370.0
ENSG00000000005.6        7.0
ENSG00000000419.13    2625.0
ENSG00000000457.14    3005.0
ENSG00000000460.17    1578.0
Name: ba295155-272e-43eb-9d6a-e4c9c392e68b, dtype: float32

## Build the combined expression matrix

We iterate through every discovered file, ensure the genes line up across samples, and concatenate the resulting vectors column-wise. Progress messages every 100 samples make it easier to monitor long runs.


In [5]:
from typing import Optional

expression_series: list[pd.Series] = []
gene_index: Optional[pd.Index] = None

total_files = len(expression_index)
for idx, path in enumerate(expression_index["path"], start=1):
    series = read_expression_table(Path(path))
    if gene_index is None:
        gene_index = series.index
    elif not series.index.equals(gene_index):
        # Align to the reference order if a file arrives with an unexpected layout.
        series = series.reindex(gene_index)
    expression_series.append(series)
    if idx % 100 == 0 or idx == total_files:
        print(f"Loaded {idx}/{total_files} samples")

expression_matrix = pd.concat(expression_series, axis=1)
expression_matrix.head()


Loaded 100/1197 samples
Loaded 200/1197 samples
Loaded 300/1197 samples
Loaded 400/1197 samples
Loaded 500/1197 samples
Loaded 600/1197 samples
Loaded 700/1197 samples
Loaded 800/1197 samples
Loaded 900/1197 samples
Loaded 1000/1197 samples
Loaded 1100/1197 samples
Loaded 1197/1197 samples


,ba295155-272e-43eb-9d6a-e4c9c392e68b,8d1641ea-7552-4d23-9298-094e0056386a,2f51534b-248b-4999-bc3f-e42a2e98332e,b321a3f5-043d-42c6-8c9d-5784d45cb85c,cafc9e36-c5f0-45df-ad03-16210ff0d870,c763a483-415e-4cb4-9cdf-4e6c31e8a9c9,7135f14b-e84f-4ebf-8d95-b2a3c843fd4d,5fd42405-ebfe-4210-bc2f-d8310e3e14ee,9ccd787f-fde1-4fe6-a11f-d6203eaf9faf,4a88d54f-c88c-4ffd-84c9-069b53f2cb28,...,ca9d1ab5-ea78-46f4-9225-8147639d013c,97a50d88-e662-43d5-9cef-2915545e8968,8a27980e-a506-4cb3-91f2-3f0e5a19acfe,fe00aecb-7b06-4e76-b999-99ee41ad20ea,2c1b1cbb-6e9f-4416-8faa-abb31d6b4e0e,967f7008-e212-4114-ab43-dc2a6295f80c,deb7967f-9339-4bb1-ae0b-81a72a472bba,574d0a5f-8cb7-4783-8d5e-b07c1b3460dc,64b12ba7-a481-4fdb-9c74-38c94c7ef3c9,74179e5e-2d3c-417e-8844-6740ea9fb2e5
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003.15,4370.0,2443.0,3508.0,6928.0,2890.0,1635.0,3456.0,1410.0,1899.0,3362.0,...,4518.0,3958.0,5469.0,2212.0,3321.0,2586.0,2127.0,8020.0,1417.0,4263.0
ENSG00000000005.6,7.0,144.0,7.0,17.0,4.0,101.0,22.0,14.0,4.0,35.0,...,10.0,18.0,9.0,0.0,0.0,286.0,0.0,22722.0,2.0,9.0
ENSG00000000419.13,2625.0,2322.0,2421.0,1812.0,4025.0,1565.0,1779.0,1431.0,2167.0,3346.0,...,1657.0,2113.0,2153.0,1184.0,3880.0,1655.0,742.0,1676.0,1138.0,2071.0
ENSG00000000457.14,3005.0,1466.0,839.0,1651.0,2769.0,1183.0,2176.0,1556.0,1516.0,2135.0,...,2009.0,1830.0,985.0,1426.0,1559.0,1723.0,911.0,1065.0,904.0,1101.0
ENSG00000000460.17,1578.0,409.0,744.0,366.0,663.0,419.0,864.0,318.0,417.0,973.0,...,739.0,931.0,1452.0,343.0,880.0,600.0,328.0,918.0,233.0,717.0


In [6]:
print(f"Expression matrix shape: {expression_matrix.shape}")
expression_matrix.iloc[:5, :5]

Expression matrix shape: (60660, 1197)


,ba295155-272e-43eb-9d6a-e4c9c392e68b,8d1641ea-7552-4d23-9298-094e0056386a,2f51534b-248b-4999-bc3f-e42a2e98332e,b321a3f5-043d-42c6-8c9d-5784d45cb85c,cafc9e36-c5f0-45df-ad03-16210ff0d870
gene_id,,,,,
ENSG00000000003.15,4370.0,2443.0,3508.0,6928.0,2890.0
ENSG00000000005.6,7.0,144.0,7.0,17.0,4.0
ENSG00000000419.13,2625.0,2322.0,2421.0,1812.0,4025.0
ENSG00000000457.14,3005.0,1466.0,839.0,1651.0,2769.0
ENSG00000000460.17,1578.0,409.0,744.0,366.0,663.0


## Persist processed outputs

The TPM matrix is saved as a gzipped TSV to keep downstream tooling simple. A companion manifest captures where each column originated and the original file sizes.


In [7]:
expression_output_path = PROCESSED_DATA_DIR / "tcga_brca_expression_tpm.tsv.gz"
manifest_output_path = PROCESSED_DATA_DIR / "expression_file_index.tsv"

expression_matrix.to_csv(expression_output_path, sep="	", compression="gzip")

manifest_df = expression_index.copy()
manifest_df["file_size_mb"] = manifest_df["path"].map(lambda p: round(p.stat().st_size / 1024 ** 2, 3))
manifest_df = manifest_df.drop(columns=["path"])
manifest_df.to_csv(manifest_output_path, sep="	", index=False)

expression_output_path, manifest_output_path


(PosixPath('/Users/lennonmccartney/Desktop/tcga-brca-multiomics-subtyping/data/processed/tcga_brca_expression_tpm.tsv.gz'),
 PosixPath('/Users/lennonmccartney/Desktop/tcga-brca-multiomics-subtyping/data/processed/expression_file_index.tsv'))

## Quick summary statistics

Verify the number of samples/genes and inspect the distribution of file sizes to catch obvious anomalies.


In [8]:
summary = pd.Series(
    {
        "n_samples": expression_matrix.shape[1],
        "n_genes": expression_matrix.shape[0],
        "min_file_size_mb": expression_index["path"].map(lambda p: p.stat().st_size / 1024 ** 2).min(),
        "max_file_size_mb": expression_index["path"].map(lambda p: p.stat().st_size / 1024 ** 2).max(),
    }
)
summary


n_samples            1197.000000
n_genes             60660.000000
min_file_size_mb        4.007236
max_file_size_mb        4.075985
dtype: float64